<a href="https://colab.research.google.com/github/ghirailghiro/GPU_Computing_Project/blob/3-first-step-gradient-computation/Gradient_First_Step.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Cuda plugin

!pip install nvcc4jupyter
%load_ext nvcc4jupyter
!nvidia-smi

# plugin for cpp sintax highlighting

!wget -O cpp_plugin.py https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
%load_ext cpp_plugin

Source files will be saved in "/tmp/tmp7axipjgv".
Sat Feb 17 16:14:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------

In [8]:
!sudo apt update
!sudo apt install -y build-essential
!sudo apt install -y libopencv-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,070 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,834 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security

In [21]:
%%writefile gradient_computation.cu
#include <opencv2/opencv.hpp>
#include <iostream>

// CUDA kernel function declaration
__global__ void printPixelValues(const unsigned char* image, int width, int height) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    int idy = threadIdx.y + blockIdx.y * blockDim.y;
    int index = idy * width + idx;

    // Ensure we don't go out of bounds
    if(idx < width && idy < height) {
        printf("Pixel (%d, %d): %d\n", idx, idy, image[index]);
    }
}

__global__ void computeGradients(float *d_image, float *d_magnitude, float *d_orientation, int width, int height) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int idy = blockIdx.y * blockDim.y + threadIdx.y;

    if (idx >= width || idy >= height) return; // Boundary check

    // Compute G_x and G_y using your chosen method
    // Skip computation for border pixels to simplify

    // Compute magnitude and orientation
    // Store results in d_magnitude and d_orientation
}



int main() {
    // Example: Load an image using OpenCV
    cv::Mat image = cv::imread("image.jpg", cv::IMREAD_GRAYSCALE);
    if(image.empty()) {
        std::cerr << "Failed to load image." << std::endl;
        return -1;
    }

    float* d_image;
    size_t imageSize = image.total() * image.elemSize();
    cudaMalloc(&d_image, imageSize);
    cudaMemcpy(d_image, image.data, imageSize, cudaMemcpyHostToDevice);

    // Assuming image dimensions are reasonable for a grid of threads
    dim3 blockSize(16, 16);
    dim3 gridSize((image.cols + blockSize.x - 1) / blockSize.x,
                  (image.rows + blockSize.y - 1) / blockSize.y);

    // Launch the kernel
    printPixelValues<<<gridSize, blockSize>>>(d_image, image.cols, image.rows);
    cudaDeviceSynchronize();

    cudaFree(d_image);
    return 0;
}


Overwriting gradient_computation.cu


In [22]:
!nvcc gradient_computation.cu -o gradient_computation `pkg-config --cflags --libs opencv4` -run


Output streaming troncato alle ultime 5000 righe.
Pixel (425, 455): 33
Pixel (426, 455): 25
Pixel (427, 455): 43
Pixel (428, 455): 24
Pixel (429, 455): 17
Pixel (430, 455): 141
Pixel (431, 455): 135
Pixel (416, 448): 32
Pixel (417, 448): 32
Pixel (418, 448): 32
Pixel (419, 448): 32
Pixel (420, 448): 32
Pixel (421, 448): 32
Pixel (422, 448): 32
Pixel (423, 448): 32
Pixel (424, 448): 29
Pixel (425, 448): 28
Pixel (426, 448): 39
Pixel (427, 448): 28
Pixel (428, 448): 40
Pixel (429, 448): 12
Pixel (430, 448): 55
Pixel (431, 448): 141
Pixel (416, 449): 32
Pixel (417, 449): 32
Pixel (418, 449): 32
Pixel (419, 449): 32
Pixel (420, 449): 32
Pixel (421, 449): 32
Pixel (422, 449): 32
Pixel (423, 449): 32
Pixel (424, 449): 33
Pixel (425, 449): 29
Pixel (426, 449): 35
Pixel (427, 449): 24
Pixel (428, 449): 36
Pixel (429, 449): 16
Pixel (430, 449): 72
Pixel (431, 449): 156
Pixel (416, 450): 32
Pixel (417, 450): 32
Pixel (418, 450): 32
Pixel (419, 450): 32
Pixel (420, 450): 32
Pixel (421, 450): 32
P

In [17]:
!./gradient_computation

Image loaded with dimensions: 910 x 607
Hello from the GPU.
